In [3]:
sc

In [21]:
dfTweets = sqlContext.read.load('samplestream_twt.jsonl', format='json')
# dfTweets.printSchema()

In [36]:
retweet_status = dfTweets.select('place.bounding_box', 'place.full_name')
retweet_status.registerTempTable('toplocations')
occurrence = sqlContext.sql('select bounding_box, full_name, count(full_name) as occurrence from toplocations group by full_name, bounding_box').dropna() 
toploc = occurrence.orderBy('occurrence', ascending=False) #order by descending order based on occurrence

In [38]:
# This command is what we use in the cluster.
toploc.rdd.saveAsTextFile('finaltoplocationsout')

## After running the code above on cluster

In [42]:
toplocations = sc.textFile("finaltoplocationsout")

In [43]:
toplocations.take(1)

[u"Row(bounding_box=Row(coordinates=[[[-95.823268, 29.522325], [-95.823268, 30.154665], [-95.069705, 30.154665], [-95.069705, 29.522325]]], type=u'Polygon'), full_name=u'Houston, TX', occurrence=5)"]

In [44]:
remove = toplocations.map(lambda x: x.replace("Row(bounding_box=Row(coordinates=[[[", ""))
remove = remove.map(lambda x: x.replace("]], type=u'Polygon'), full_name=u'", ","))
remove = remove.map(lambda x: x.replace(", occurrence=", ";"))
final = remove.map(lambda x: x.replace(")", ", NULL").replace("],", ';').replace('[', '').replace(']', '').replace("'", ''))

final.take(2)

[u'-95.823268, 29.522325; -95.823268, 30.154665; -95.069705, 30.154665; -95.069705, 29.522325;Houston, TX;5, NULL',
 u'-74.026675, 40.683935; -74.026675, 40.877483; -73.910408, 40.877483; -73.910408, 40.683935;Manhattan, NY;5, NULL']

In [45]:
def mapper(k1v1s): 
    for k1v1 in k1v1s:
        k = k1v1.split(';')
        x_co = float(k[0].split(',')[0]) + float(k[1].split(',')[0])+ float(k[2].split(',')[0])+float(k[3].split(',')[0])
        x_co = x_co/4
        y_co = float(k[0].split(',')[1]) + float(k[1].split(',')[1])+ float(k[2].split(',')[1])+float(k[3].split(',')[1])
        y_co = y_co/4
        yield (round(y_co,6), round(x_co, 6), k[4], int(k[5].split(',')[0]))

coordinates = final.mapPartitions(mapper)
coordinates.take(5) #count()

[(29.838495, -95.446486, u'Houston, TX', 5),
 (40.780709, -73.968542, u'Manhattan, NY', 5),
 (43.056722, -87.967292, u'Milwaukee, WI', 1),
 (32.189391, -110.915573, u'Tucson, AZ', 1),
 (48.088757, -122.164528, u'Marysville, WA', 1)]

In [34]:
import csv

def writeRecords(records):
    with open('coordinates.csv', 'w') as csvfile:
        field = ['latitude', 'longitude', 'location','count']
        writer = csv.DictWriter(csvfile, fieldnames=field)

        writer.writeheader()
        for record in records:
            writer.writerow({'latitude': record[0], 'longitude': record[1], 'location': record[2], 'count': record[3]})

In [35]:
coordinates = coordinates.toLocalIterator()
writeRecords(coordinates)